In [3]:
pip install --upgrade nbeats-pytorch

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

# Veri setini yükleme ve hazırlama
df = pd.read_excel("rainfall-and-daily-consumption-data-on-istanbul-dams.xlsx")
df = df[['Tarih', 'İstanbul günlük tüketim(m³/gün)']]
df = df.set_index("Tarih")
df.index = pd.to_datetime(df.index)
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'] // 100
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'].astype(float)
df = np.log(df)

# Eğitim ve test setlerini oluşturma
train_size = int(len(df) * 0.80)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Verileri ölçeklendirme
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# Model oluşturma
def build_nbeats_model(input_shape, horizon):
    input_layer = Input(shape=(input_shape,))
    x = input_layer
    for _ in range(4):
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.2)(x)
    forecast = Dense(horizon)(x)
    model = Model(inputs=input_layer, outputs=forecast)
    return model

input_shape = train_scaled.shape[1]
horizon = 1  # Tek bir adım öngörü
nbeats_model = build_nbeats_model(input_shape, horizon)
nbeats_model.compile(optimizer='adam', loss='mse')

# Modeli eğitme
history = nbeats_model.fit(train_scaled[:-1], train_scaled[1:], epochs=50, batch_size=32, verbose=1, validation_data=(test_scaled[:-1], test_scaled[1:]))

# Test seti üzerinde tahmin yapma
predictions_scaled = nbeats_model.predict(test_scaled[:-1])
predictions = scaler.inverse_transform(predictions_scaled)

# Hataları hesaplama
mse = mean_squared_error(test['İstanbul günlük tüketim(m³/gün)'].iloc[1:], predictions)
print("Mean Squared Error (MSE): {:.4f}".format(mse))

rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE): {:.4f}".format(rmse))

mae = mean_absolute_error(test['İstanbul günlük tüketim(m³/gün)'].iloc[1:], predictions)
print("Mean Absolute Error (MAE): {:.4f}".format(mae))

mape = mean_absolute_percentage_error(test['İstanbul günlük tüketim(m³/gün)'].iloc[1:], predictions)
print("Mean Absolute Percentage Error (MAPE): {:.4f}".format(mape))


Epoch 1/50
118/118 [==============================] - 1s 5ms/step - loss: 0.0172 - val_loss: 0.0031
Epoch 2/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0068 - val_loss: 0.0061
Epoch 3/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0061 - val_loss: 0.0184
Epoch 4/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0057 - val_loss: 0.0171
Epoch 5/50
118/118 [==============================] - 0s 4ms/step - loss: 0.0056 - val_loss: 0.0094
Epoch 6/50
118/118 [==============================] - 0s 4ms/step - loss: 0.0056 - val_loss: 0.0062
Epoch 7/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0053 - val_loss: 0.0051
Epoch 8/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0051 - val_loss: 0.0123
Epoch 9/50
118/118 [==============================] - 0s 4ms/step - loss: 0.0049 - val_loss: 0.0087
Epoch 10/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0050 - val_loss: 0.0069